In [2]:
from datetime import date

import numpy as np
import pandas as pd
import yaml

In [3]:
from sqlalchemy import create_engine

with open('./config.yml', 'r') as f:
        config = yaml.safe_load(f)
        config_ryf = config['RAPIDO_Y_FURIOSO']
        config_etl = config['ETL']

url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@{config_ryf['host']}:"
        f"{config_ryf['port']}/{config_ryf['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
        f"{config_etl['port']}/{config_etl['dbname']}")

ryf = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

In [4]:
dim_cliente = pd.read_sql_table('cliente', ryf)

In [5]:
dim_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cliente_id       27 non-null     int64  
 1   nit_cliente      27 non-null     object 
 2   nombre           27 non-null     object 
 3   email            27 non-null     object 
 4   direccion        27 non-null     object 
 5   telefono         27 non-null     object 
 6   nombre_contacto  27 non-null     object 
 7   ciudad_id        27 non-null     int64  
 8   tipo_cliente_id  27 non-null     int64  
 9   activo           27 non-null     bool   
 10  coordinador_id   3 non-null      float64
 11  sector           27 non-null     object 
dtypes: bool(1), float64(1), int64(3), object(7)
memory usage: 2.5+ KB


In [6]:
dim_cliente.describe(include='all')

,cliente_id,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,ciudad_id,tipo_cliente_id,activo,coordinador_id,sector
count,27.000000,27,27,27,27,27,27,27.000000,27.0,27,3.0,27
unique,NaN,27,27,1,1,1,1,NaN,NaN,1,NaN,3
top,NaN,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,NaN,NaN,True,NaN,salud
freq,NaN,1,1,27,27,27,27,NaN,NaN,27,NaN,17
mean,14.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.259259,1.0,NaN,2.0,NaN
std,7.937254,NaN,NaN,NaN,NaN,NaN,NaN,1.347151,0.0,NaN,1.0,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,NaN,1.0,NaN
25%,7.500000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,NaN,1.5,NaN
50%,14.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,NaN,2.0,NaN
75%,20.500000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,NaN,2.5,NaN


In [7]:
dim_cliente = dim_cliente.filter(['cliente_id', 'nit_cliente', 'nombre', 'direccion', 'email', 'telefono'])

In [8]:
dim_cliente.head()

,cliente_id,nit_cliente,nombre,direccion,email,telefono
0,1,25,Cliente 2,Calle 100 No 25-18,algo.com,327-00000
1,2,123,Cliente 1,Calle 100 No 25-18,algo.com,327-00000
2,6,24390-3,CLINICA DEPORTIVA DEL SUR,Calle 100 No 25-18,algo.com,327-00000
3,19,8301821,HOSPITAL ORTOPEDICO DE COLOMBIA,Calle 100 No 25-18,algo.com,327-00000
4,8,5017350-8,CLINICA NEFROLOGOS DE CALI,Calle 100 No 25-18,algo.com,327-00000


In [9]:
dim_cliente.to_sql('dim_cliente', etl_conn, if_exists='replace',index_label='key_dim_cliente')

27